<a href="https://colab.research.google.com/github/chaufe/testcases/blob/main/colab/OpenRoad__excessive_buffering_on_high_fanout_net_when_resizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Testcase to show excessive buffering in OpenRoad on high-fanout nets**

### Environment and Software Setup
- setup environment
- install conda-eda, GF180MCU PDK and EDA software
- install and patch helper software

In [ ]:
#@title Install a few packages required later-on { display-mode: "form" }
!dpkg --configure -a
!apt install python3-click

In [ ]:
#@title Install Anaconda environment {display-mode: "form"}
CONDA_PREFIX = '/content/eda'
%env CONDA_PREFIX = {CONDA_PREFIX}
!echo 'Will install CONDA EDA to {CONDA_PREFIX}'
!test -e Install.sh || wget -O Install.sh https://github.com/proppy/conda-eda/releases/download/v0.0-1445-gdbbed53/digital.gf180mcuc-0-Linux-x86_64.sh
!test -d {CONDA_PREFIX} || bash Install.sh -b -p {CONDA_PREFIX}

In [ ]:
#@title Update conda packages {display-mode: "form"}
!{CONDA_PREFIX}/bin/conda update --yes --all

In [ ]:
#@title Install EDA software { display-mode: "form" }

# use hard-coded packages as version resolution does not work as expected
for pkg in [
    'https://anaconda.org/LiteX-Hub/openroad/2.0_9365_g327a5f003/download/linux-64/openroad-2.0_9365_g327a5f003-20230709_210322.tar.bz2',
]:
  print("Installing " + pkg)
  !{CONDA_PREFIX}/bin/conda install --yes {pkg}

In [ ]:
#@title Setup remaining variables { display-mode: 'form'}
%env CONDA_PREFIX = {CONDA_PREFIX}
import os
%env PATH = {CONDA_PREFIX + '/bin:' + os.getenv('PATH')}
%env PDK_ROOT = {CONDA_PREFIX + '/share/pdk'}
%env PDK = gf180mcuC
%env IO_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io'}
%env SRAM_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_ip_sram'}
!echo $PYTHONPATH

In [ ]:
#@title Create patch file for GF180MCU IO IP { display-mode: "form" }
#!wget https://raw.githubusercontent.com/chaufe/testcases/main/patches/gf180mcu_fd_io.lef.patch
%%writefile "gf180mcu_fd_io.lef.patch"
--- eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/lef/gf180mcu_fd_io.lef	2023-02-13 13:43:43.000000000 +0100
+++ eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/lef/gf180mcu_fd_io.lef	2023-03-30 10:19:34.792862000 +0200
@@ -3,6 +3,20 @@
   DIVIDERCHAR "/" ;
   BUSBITCHARS "[]" ;
+
+SITE GF_IO_Site
+  SYMMETRY X Y ;
+  CLASS PAD ;
+  SIZE 1 BY 350 ;
+END GF_IO_Site
+
+SITE GF_COR_Site
+  SYMMETRY X Y ;
+  CLASS PAD ;
+  SIZE 355 BY 355 ;
+END GF_COR_Site
+
+
 MACRO gf180mcu_fd_io__asig_5p0
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__asig_5p0 ;
   ORIGIN 0.000 0.000 ;
@@ -815,5 +829,5 @@

 MACRO gf180mcu_fd_io__bi_24t
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__bi_24t ;
   ORIGIN 0.000 0.000 ;
@@ -1669,5 +1683,5 @@

 MACRO gf180mcu_fd_io__bi_t
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__bi_t ;
   ORIGIN 0.000 0.000 ;
@@ -3807,5 +3821,5 @@

 MACRO gf180mcu_fd_io__dvdd
-  CLASS PAD POWER ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__dvdd ;
   ORIGIN 0.000 0.000 ;
@@ -4600,5 +4614,5 @@

 MACRO gf180mcu_fd_io__dvss
-  CLASS PAD POWER ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__dvss ;
   ORIGIN 0.000 0.000 ;
@@ -7609,5 +7623,5 @@

 MACRO gf180mcu_fd_io__in_c
-  CLASS PAD INPUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__in_c ;
   ORIGIN 0.000 0.000 ;
@@ -8416,5 +8430,5 @@

 MACRO gf180mcu_fd_io__in_s
-  CLASS PAD INPUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__in_s ;
   ORIGIN 0.000 0.000 ;


In [ ]:
#@title Patch GF180MCU IP { display-mode: "form" }
!patch -p0 < gf180mcu_fd_io.lef.patch

### Create Testcase

In [64]:
#@title Write testcase RTL { display-mode: "form" }
%%writefile "testcase.v"
module testcase (input clk,
                 input din,
                 input rst,
                 output wire dout);

   parameter FLOPS = 500;

   wire clk_int;
   wire din_int;
   wire rst_int;
   wire dout_int;


   // pads
   (* keep *) gf180mcu_fd_io__bi_t pad_dout(
                                            .PU(1'b0), .PD(1'b0),
                                            .OE(1'b1), .IE(1'b0),
                                            .PDRV0(1'b0), .PDRV1(1'b0),
                                            .CS(1'b0), .SL(1'b0),
                                            .PAD(dout),
                                            .A(dout_int),
                                            .Y()
                                            );

   (* keep *) gf180mcu_fd_io__in_s pad_clk(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(clk),
                                           .Y(clk_int)
                                           );

   (* keep *) gf180mcu_fd_io__in_s pad_din(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(din),
                                           .Y(din_int)
                                           );

   (* keep *) gf180mcu_fd_io__in_s pad_rst(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(rst),
                                           .Y(rst_int/*_unbuf*/)
                                           );

// gf180mcu_fd_sc_mcu7t5v0__buf_4 buf_i(.I(rst_int_unbuf), .Z(rst_int)) ;

  wire [FLOPS-1:0] flop_q;

  gf180mcu_fd_sc_mcu7t5v0__sdffq_1 flops [FLOPS-1:0] (
    .CLK({FLOPS{clk_int}}),
    .SE({FLOPS{rst_int}}),
    .SI({FLOPS{1'b0}}),
    .D({flop_q[FLOPS-2:0], din_int}),
    .Q(flop_q),
  );

  gf180mcu_fd_sc_mcu7t5v0__sdffq_1 flop_xor (
    .CLK(clk_int),
    .SE(rst_int),
    .SI(1'b0),
    .D(^{flop_q/*,sram_q*/}),
    .Q(dout_int),
  );

endmodule


Overwriting testcase.v


In [52]:
#@title Write config.tcl to control OpenLANE { display-mode: "form" }
%%writefile "config.tcl"

set ::env(DESIGN_NAME) "testcase"

set ::env(VERILOG_FILES) [list "testcase.v" ]

set ::env(EXTRA_LIBS) [list \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__tt_025C_5v00.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_n40C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_125C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ss_125C_4v50.lib \
  $::env(PDK_ROOT)/gf180mcuC/libs.ref/gf180mcu_fd_sc_mcu7t5v0/liberty/gf180mcu_fd_sc_mcu7t5v0__tt_025C_5v00.lib \
]

set ::env(EXTRA_LEFS) [list \
  $::env(PDK_ROOT)/gf180mcuC/libs.ref/gf180mcu_fd_sc_mcu7t5v0/lef/gf180mcu_fd_sc_mcu7t5v0.lef \
]

set ::env(EXTRA_GDS_FILES) [list \
  $::env(IO_LIB_PATH)/gds/gf180mcu_fd_io.gds \
]

set ::env(DESIGN_IS_CORE) 1
set ::env(FP_PDN_CORE_RING) 1

set ::env(GRT_ALLOW_CONGESTION) 1
set ::env(DRT_OPT_ITERS) 10

set ::env(DIODE_INSERTION_STRATEGY) 0

set ::env(USE_GPIO_PADS) 1
set ::env(GPIO_PADS_LEF) $::env(IO_LIB_PATH)/lef/gf180mcu_fd_io.lef
unset ::env(GPIO_PADS_VERILOG)
set ::env(FP_IO_MODE) 1

set ::env(FP_PDN_UPPER_LAYER) Metal5
set ::env(FP_PDN_LOWER_LAYER) Metal4
set ::env(FP_PDN_RAILS_LAYER) Metal1
set ::env(FP_PDN_CORE_RING_VWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_HWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_VSPACING) 1.0
set ::env(FP_PDN_CORE_RING_HSPACING) 1.0

set ::env(SYNTH_NO_FLAT) 0
set ::env(SYNTH_SHARE_RESOURCES) 1

set ::env(CLOCK_TREE_SYNTH) 1
set ::env(CLOCK_PORT) "clk"
set ::env(CLOCK_PERIOD) 100

set ::env(FP_SIZING) absolute
set ::env(DIE_AREA) "0 0 1460 1460"
set ::env(CORE_AREA) "370 370 1090 1090"

set ::env(PL_TARGET_DENSITY) 0.65

# MAGIC GDS-out controls
## ... try GDS with hierarchy
set ::env(MAGIC_DISABLE_HIER_GDS) 0
set ::env(MAGIC_GENERATE_LEF) 0
set ::env(MAGIC_GENERATE_MAGLEF) 0

Overwriting config.tcl


In [10]:
#@title Create macro config for dummy SRAM { display-mode: "form" }
%%writefile 'macro.cfg'
sram_i  370 370 FS ;

Writing macro.cfg


In [73]:
#@title Use updated ioplace script to create pad ring { display-mode: "form" }
%%writefile "eda/share/openlane/scripts/openroad/ioplacer.tcl"
# Copyright 2020-2022 Efabless Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
source $::env(SCRIPTS_DIR)/openroad/common/io.tcl
read

if { [info exists ::env(CONTEXTUAL_IO_FLAG)] } {
	read_lef $::env(placement_tmpfiles)/top_level.lef
}

if {$::env(FP_IO_HLENGTH) != "" && $::env(FP_IO_HLENGTH) != ""} {
	set_pin_length -hor_length $::env(FP_IO_HLENGTH) \
		-ver_length $::env(FP_IO_VLENGTH)
}

if {$::env(FP_IO_HEXTEND) != "0" && $::env(FP_IO_VEXTEND) != "0"} {
	set_pin_length_extension -hor_extension $::env(FP_IO_HEXTEND) \
		-ver_extension $::env(FP_IO_VEXTEND)
}

if {$::env(FP_IO_VTHICKNESS_MULT) != "" && $::env(FP_IO_HTHICKNESS_MULT) != ""} {
	set_pin_thick_multiplier -hor_multiplier $::env(FP_IO_HTHICKNESS_MULT) \
		-ver_multiplier $::env(FP_IO_VTHICKNESS_MULT)
}

set arg_list [list]
if { $::env(FP_IO_MODE) == 1 } {
	lappend arg_list -random
}

if { $::env(FP_IO_MIN_DISTANCE) != "" } {
	lappend arg_list -min_distance $::env(FP_IO_MIN_DISTANCE)
}

set HMETAL $::env(FP_IO_HLAYER)
set VMETAL $::env(FP_IO_VLAYER)

place_pins {*}$arg_list \
	-random_seed 42 \
	-hor_layers $HMETAL \
	-ver_layers $VMETAL

# create io-ring
make_io_sites -horizontal_site GF_IO_Site -vertical_site GF_IO_Site -corner_site GF_COR_Site -offset 0
place_pad -master gf180mcu_fd_io__in_s   -row IO_NORTH -location  400.0 pad_dout
place_pad -master gf180mcu_fd_io__in_s   -row IO_NORTH -location  700.0 pad_clk
place_pad -master gf180mcu_fd_io__dvss   -row IO_NORTH -location 1000.0 pad_dvss1

place_pad -master gf180mcu_fd_io__bi_24t -row IO_EAST  -location  400.0 pad_rst
place_pad -master gf180mcu_fd_io__in_s   -row IO_EAST  -location  700.0 pad_din
place_pad -master gf180mcu_fd_io__dvdd   -row IO_EAST  -location 1000.0 pad_dvdd1

place_pad -master gf180mcu_fd_io__dvdd   -row IO_WEST  -location  400.0 pad_dvdd2
place_pad -master gf180mcu_fd_io__dvss   -row IO_WEST  -location  700.0 pad_dvss2
place_pad -master gf180mcu_fd_io__dvdd   -row IO_WEST  -location 1000.0 pad_dvdd3

place_pad -master gf180mcu_fd_io__dvss   -row IO_SOUTH -location  400.0 pad_dvss3
place_pad -master gf180mcu_fd_io__dvdd   -row IO_SOUTH -location  700.0 pad_dvdd4
place_pad -master gf180mcu_fd_io__dvss   -row IO_SOUTH -location 1000.0 pad_dvss4

place_corners gf180mcu_fd_io__cor
foreach io_row { IO_EAST IO_WEST IO_SOUTH IO_NORTH } {
    place_io_fill -row $io_row gf180mcu_fd_io__fill10 gf180mcu_fd_io__fill5 gf180mcu_fd_io__fill1
}

connect_by_abutment

# work-around for pin-access error message

# get db/chip/block
set db [ord::get_db]
set chip [$db getChip]
set block [$chip getBlock]
set dbu_m [$block getDbUnitsPerMicron]

# mark all nets connected to pins (terminals) as special-net so OpenROAD will not try to route it
foreach bterm [$block getBTerms] {
  if {$bterm eq "NULL"} {
      error "Could not get terminals of block"
  } else {
    set net [$bterm getNet]
    if {$net eq "NULL"} {
      error "Block terminal [$bterm getName] returned NULL net"
    } else {
      puts "Setting net [$net getName] (connected to block terminal [$bterm getName]) to special-net"
      $net setSpecial
    }
  }
}

write


Overwriting eda/share/openlane/scripts/openroad/ioplacer.tcl


In [ ]:
#@title Create modified version of resizer.tcl { display-mode: "form" }
%%writefile eda/share/openlane/scripts/openroad/resizer.tcl
# Copyright 2020-2022 Efabless Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
source $::env(SCRIPTS_DIR)/openroad/common/io.tcl
read -override_libs "$::env(RSZ_LIB)"

unset_propagated_clock [all_clocks]

# set rc values
source $::env(SCRIPTS_DIR)/openroad/common/set_rc.tcl

# estimate wire rc parasitics
estimate_parasitics -placement

# work-around excessive buffering by adding one buffer to high-fanout net prior to resize
if {[info exists ::env(WORKAROUND)] && ($::env(WORKAROUND) eq "enabled")} {
  make_net rst_int_buffered
  make_inst rst_int_buf gf180mcu_fd_sc_mcu7t5v0__buf_4
  foreach pin [get_pins -of_objects [get_nets rst_int] -filter "direction == input"] {
    disconnect_pin rst_int $pin
    connect_pin rst_int_buffered $pin
  }
  connect_pin rst_int_buffered rst_int_buf/Z
  connect_pin rst_int rst_int_buf/I
  # place new cell rst_inst_buf
  global_placement -incremental
}

report_design_area

# set don't touch nets
source $::env(SCRIPTS_DIR)/openroad/common/resizer.tcl
set_dont_touch_rx "$::env(RSZ_DONT_TOUCH_RX)"

# set don't use cells
if { [info exists ::env(DONT_USE_CELLS)] } {
    set_dont_use $::env(DONT_USE_CELLS)
}

if { [info exists ::env(PL_RESIZER_BUFFER_INPUT_PORTS)] && $::env(PL_RESIZER_BUFFER_INPUT_PORTS) } {
    buffer_ports -inputs
}

if { [info exists ::env(PL_RESIZER_BUFFER_OUTPUT_PORTS)] && $::env(PL_RESIZER_BUFFER_OUTPUT_PORTS) } {
    buffer_ports -outputs
}
# Resize
if { [info exists ::env(PL_RESIZER_MAX_WIRE_LENGTH)] && $::env(PL_RESIZER_MAX_WIRE_LENGTH) } {
    repair_design -max_wire_length $::env(PL_RESIZER_MAX_WIRE_LENGTH) \
        -slew_margin $::env(PL_RESIZER_MAX_SLEW_MARGIN) \
        -cap_margin $::env(PL_RESIZER_MAX_CAP_MARGIN)
} else {
    repair_design -slew_margin $::env(PL_RESIZER_MAX_SLEW_MARGIN) \
        -cap_margin $::env(PL_RESIZER_MAX_CAP_MARGIN)
}
report_design_area
if { $::env(PL_RESIZER_REPAIR_TIE_FANOUT) == 1} {
    # repair tie lo fanout
    repair_tie_fanout -separation $::env(PL_RESIZER_TIE_SEPERATION) [lindex $::env(SYNTH_TIELO_PORT) 0]/[lindex $::env(SYNTH_TIELO_PORT) 1]
    # repair tie hi fanout
    repair_tie_fanout -separation $::env(PL_RESIZER_TIE_SEPERATION) [lindex $::env(SYNTH_TIEHI_PORT) 0]/[lindex $::env(SYNTH_TIEHI_PORT) 1]
}

report_floating_nets -verbose

source $::env(SCRIPTS_DIR)/openroad/common/dpl_cell_pad.tcl

detailed_placement

if { [info exists ::env(PL_OPTIMIZE_MIRRORING)] && $::env(PL_OPTIMIZE_MIRRORING) } {
    optimize_mirroring
}

if { [catch {check_placement -verbose} errmsg] } {
    puts stderr $errmsg
    exit 1
}

unset_dont_touch_rx "$::env(RSZ_DONT_TOUCH_RX)"

write

# Run post design optimizations STA
estimate_parasitics -placement

### Run Testcase

In [20]:
#@title Enable or disable work-around { display-mode: "form" }
import ipywidgets as widgets
import IPython

workaround = widgets.RadioButtons(
    options=['enabled', 'disabled'],
    value='disabled',
    disabled=False
)
print('Enable or disable work-around?:')
IPython.display.display(workaround)

Enable or disable work-around?:


RadioButtons(index=1, options=('enabled', 'disabled'), value='disabled')

In [ ]:
#@title Run OpenLANE { display-mode: "form" }
%env WORKAROUND = {workaround.value}
%env PYTHONPATH = /usr/lib/python310.zip;/usr/lib/python3.10:/usr/lib/python3.10/lib-dynload:/usr/local/lib/python3.10/dist-packages:/usr/lib/python3/dist-packages:/usr/local/lib/python3.10/dist-packages/IPython/extensions
!flow.tcl -design . -to placement
!grep -H -m1 -A7 "Design area" `ls runs/*/logs/placement/*-resizer.log | tail -n 1`

In [ ]:
#@title Package issue { display-mode: "form" }
import pathlib
STEP='8'
odb = sorted(pathlib.Path('/content/runs/').glob(f'*/tmp/*/{STEP}-*.odb'))[-1].resolve()

%env PYTHONPATH = /content/eda/lib/python3.7:/content/eda/lib/python37.zip:/content/eda/lib/python3.7/lib-dynload:/content/eda/lib/python3.7/site-packages
!python3.7 /content/eda/share/openlane/scripts/or_issue.py\
    --tool openroad\
    --script /content/eda/share/openlane/scripts/openroad/resizer.tcl {odb}

!zip -9 -r _build.coredump_resizer.zip _build